Pipeline API是用于对资产数据进行横截面分析的强大工具。它使我们能够定义一组关于多个数据输入的计算，并一次分析大量股票。Pipeline API的一些常见用途包括：
+ 根据过滤规则选择资产
+ 根据评分函数对资产进行排名
+ 计算投资组合分配

首先，我们导入Pipeline类并创建一个返回空`pipeline`的函数。将`pipeline`定义放入一个函数中，可以帮助我们随着`pipeline`复杂度的增长保持组织的有序性。在Research和IDE之间传输数据管道时，这特别有用。

In [1]:
# Pipeline class
from zipline.pipeline import Pipeline

def make_pipeline():
    # Create and return an empty Pipeline
    return Pipeline()

要将输出添加到`pipeline`中，我们需要包含对数据集的引用，并指定要对这些数据执行的计算。 例如，我们将添加对`USEquityPricing`数据集中的`close`列的引用。 然后，我们可以将输出定义为该列中的最新值，如下所示：

In [2]:
# Import Pipeline class and USEquityPricing dataset
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing

def make_pipeline():
    # Get latest closing price
    close_price = USEquityPricing.close.latest

    # Return Pipeline containing latest closing price
    return Pipeline(
        columns={
            'close_price': close_price,
        }
    )

In [3]:
from zipline.pipeline.fundamentals import Fundamentals

Pipeline API还提供了一些内置的计算，其中一些计算是在数据的尾随窗口上计算的。例如，以下代码导入数据列rating并将输出定义为rating的3天移动平均数：

In [4]:
# Import Pipeline class and datasets
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing

# Import built-in moving average calculation
from zipline.pipeline.factors import SimpleMovingAverage


def make_pipeline():
    # Get latest closing price
    close_price = USEquityPricing.close.latest

    # Calculate 3 day average of bull_minus_bear scores
    rating_score = SimpleMovingAverage(
        inputs=[Fundamentals.rating],
        window_length=3,
    )

    # Return Pipeline containing close_price
    # and sentiment_score
    return Pipeline(
        columns={
            'close_price': close_price,
            'rating_score': rating_score,
        }
    )

## 总体选择

制定策略的一个重要部分是定义投资组合中进行交易的一组资产。我们通常将这组资产称为我们的交易总体。

交易范围应尽可能大，同时也要排除不适合投资组合的资产。例如，我们想排除流动性不强或难以交易的股票。`QTradableStocksUS`总体提供这个功能。可以使用`pipeline`构造函数的`screen`参数将`QTradableStocksUS`设置为交易范围：

In [5]:
# Import Pipeline class and datasets
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing

# Import built-in moving average calculation
from zipline.pipeline.factors import SimpleMovingAverage

# Import built-in trading universe
from zipline.pipeline.builtin import QTradableStocksUS


def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest

    # Calculate 3 day average of bull_minus_bear scores
    rating_score = SimpleMovingAverage(
        inputs=[Fundamentals.rating],
        window_length=3,
    )

    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
            'rating_score': rating_score,
        },
        screen=base_universe
    )

现在`pipeline`定义已经完成，可以使用`run_pipeline`在特定的时间段内执行它。输出将是按日期和资产索引的`pandas.DataFrame`，其中的列对应于我们添加到管道定义中的输出：

In [6]:
# Import run_pipeline method
from zipline.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    '2017-01-01',
    '2018-04-30'
)

# Display last 10 rows
pipeline_output.tail(10)

close_price  rating_score
2018-04-27 00:00:00+00:00 恒润股份(603985)        32.14           NaN
                          康德莱(603987)         12.21           NaN
                          中电电机(603988)        26.76           NaN
                          艾华集团(603989)        35.17           4.0
                          麦迪科技(603990)        40.16           NaN
                          至正股份(603991)        25.65           NaN
                          洛阳钼业(603993)         7.57           4.0
                          中新科技(603996)        16.46           NaN
                          继峰股份(603997)        12.00           4.0
                          读者传媒(603999)         7.08           NaN

在下一课中，我们将使用正式确定的策略，以选择要交易的资产。 然后，我们将使用因子分析工具来评估我们策略对历史数据的预测能力。